In [ ]:
! pip install apache-beam

In [ ]:
import apache_beam as beam
import logging
import requests
from urllib.error import HTTPError


In [ ]:


class callAPI(beam.DoFn):
    def process(self, element):
        fro = element.get("from")
        to = element.get("to")

        url = 'http://127.0.0.1:8000/content?from={}&to={}'.format(fro, to)
        res = requests.get(url)
        data  = res.json()
        yield data


In [ ]:
with beam.Pipeline() as p:
    data = p | beam.Create(values= [{'from': 1, 'to': 100}])
    ap_response = data | beam.ParDo(callAPI())
    ap_response | "Print results" >> beam.Map(print)